In [5]:
import cv2

In [ ]:
# Available backend options are: "jax", "torch", "tensorflow".
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
	
import keras

@keras.saving.register_keras_serializable()
def scaling(x, scale=1.0):
	return x * scale

model = keras.saving.load_model(
	"hf://logasja/FaceNet512",
	custom_objects={"scaling": scaling}
)

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 14644.92it/s]


In [ ]:
import os
os.environ["KERAS_BACKEND"] = "jax"

import keras

# Define and register the custom scaling function
@keras.saving.register_keras_serializable()
def scaling(x, scale=1.0):
	return x * scale

# Pass custom_objects to load_model so it can resolve the custom function
model = keras.saving.load_model(
	"hf://logasja/FaceNet512",
	custom_objects={"scaling": scaling}
)

In [8]:
signature = model.predict(
    cv2.resize(
        cv2.imread("test.jpg"),
        (160, 160)
    )[None, ...]
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step


In [11]:
signature

array([[  8.582854  ,  -2.196429  ,   6.553374  ,   0.03782938,
          7.628533  ,  -9.782125  ,  -0.9322874 ,  -3.5082989 ,
          0.33592662,  -3.8825958 , -13.541861  ,  -5.007911  ,
        -12.296165  ,  -2.5035894 ,  12.824289  ,  -6.9545693 ,
          0.14934239,  -5.72149   ,   2.4672978 ,   8.194644  ,
          6.3512273 ,   3.0296693 ,  -6.754753  ,   2.9326391 ,
          2.210091  ,   0.6931978 ,  14.894694  ,   0.5971914 ,
          4.0478187 , -15.357883  ,  -1.8622705 ,   8.219791  ,
          0.89453644,   3.912957  ,   7.7641597 ,   0.3221865 ,
         -9.176463  ,   6.1724815 ,  -4.554122  ,   3.0853841 ,
          1.4770647 ,   1.0412139 ,   0.30903846, -14.953159  ,
         -4.8707204 ,  -6.9057674 ,   6.9293675 ,  -3.5773947 ,
         -6.962186  ,   4.412541  ,  16.346287  ,   9.3729105 ,
          7.7063155 ,   9.446874  ,   2.1617577 ,  -7.182056  ,
          6.5661774 ,   9.184706  ,   2.7221334 ,   4.3038454 ,
         -5.068271  , -11.527227  ,   7.

In [ ]:
# Access camera
def setCamera(index, width=640, height=480, fps=30):
    cap = cv2.VideoCapture(index)
    
    # Set the camera resolution (optional)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
    
    # Set the camera frame rate (optional)
    cap.set(cv2.CAP_PROP_FPS, fps)
    
    return cap

# change the camera index if necessary
# Common indices are 0 for the default camera, 1 for an external camera, etc.
def getCamera(index=1, max_attempt=3):
    while max_attempt:
        print(f"Opening Camera {index}...")
        cap = setCamera(index)
        if not cap.isOpened():
            print(f"Camera {index} is not Available!")
            index+=1
            max_attempt-=1
        else:
            print(f"Camera {index} is Available!")
            return cap
    
    if not max_attempt:
        print("Opening Default Camera")
        cap = setCamera(index=0)
        if not cap.isOpened():
            print("Default Camera is not Available!")
            exit()
        return cap

In [ ]:
# Access camera
cap = getCamera()

# Check if the video capture has been initialized correctly
if not cap.isOpened():
    print("Error: Could not open video device.")
    exit()

while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        print("Could not read frame.")
        break
    
    # Display the frame in a window
    cv2.imshow("Camera Feed", frame)
    
    # Wait for 1 ms and check if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
# Release the video capture object and close all OpenCV windows
# Note: The video capture object is released and all OpenCV windows are closed
# when the loop is exited, either by pressing 'q' or if an error occurs.
# This ensures that resources are cleaned up properly.
cv2.destroyAllWindows()
cap.release()